In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import resample
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import Embedding, Bidirectional, SimpleRNN, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


2023-10-20 00:39:58.055149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 00:39:58.751803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-10-20 00:39:58.751888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [3]:
df = pd.read_csv('../Data/CleanedEcommerce.csv')

In [10]:
df = df.dropna()

In [11]:
text = 'desc'
label = 'label'
num_classes = df[label].nunique()

In [12]:
num_classes

4

In [13]:
label_encoder = LabelEncoder()
df[label] = label_encoder.fit_transform(df[label])

In [14]:
X = df[text]
y = df[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify =y)

In [15]:
max_words = 1000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)


In [16]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
max_sequence_length = 300  # You can adjust this to your desired sequence length
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [17]:
y_train.shape

(22240,)

In [22]:

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_sequence_length))
model.add(Bidirectional(SimpleRNN(16, return_sequences=True)))  # Bidirectional SimpleRNN layer
model.add(Bidirectional(SimpleRNN(16, return_sequences=True)))  # Bidirectional SimpleRNN layer
model.add(SimpleRNN(16))  # SimpleRNN layer
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))  # L2 regularization
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.04, patience=10, mode='max', restore_best_weights=True)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(X_train, y_train, epochs=30, batch_size=256, validation_split=0.2,callbacks=[early_stopping])

Epoch 1/30
70/70 [==============================] - 38s 511ms/step - loss: 1.4156 - accuracy: 0.4440 - val_loss: 0.9729 - val_accuracy: 0.7010
Epoch 2/30
70/70 [==============================] - 35s 501ms/step - loss: 0.8705 - accuracy: 0.7232 - val_loss: 0.7259 - val_accuracy: 0.7556
Epoch 3/30
70/70 [==============================] - 35s 504ms/step - loss: 0.6888 - accuracy: 0.7973 - val_loss: 0.5893 - val_accuracy: 0.8476
Epoch 4/30
70/70 [==============================] - 35s 504ms/step - loss: 0.5535 - accuracy: 0.8768 - val_loss: 0.4791 - val_accuracy: 0.8898
Epoch 5/30
70/70 [==============================] - 35s 498ms/step - loss: 0.4805 - accuracy: 0.8987 - val_loss: 0.4381 - val_accuracy: 0.8984
Epoch 6/30
70/70 [==============================] - 34s 485ms/step - loss: 0.4040 - accuracy: 0.9206 - val_loss: 0.4140 - val_accuracy: 0.8988
Epoch 7/30
70/70 [==============================] - 34s 483ms/step - loss: 0.3560 - accuracy: 0.9323 - val_loss: 0.3980 - val_accuracy: 0.9047

In [25]:
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
f1 = f1_score(y_test, y_pred, average='weighted')
classification_rep = classification_report(y_test, y_pred)

print(f'Weighted F1 Score: {f1}')
print("Classification Report :\n",classification_rep)

174/174 [==============================] - 12s 66ms/step
Weighted F1 Score: 0.8888784497718066
Classification Report :
               precision    recall  f1-score   support

           0       0.93      0.88      0.91      1251
           1       0.81      0.95      0.87      1135
           2       0.90      0.78      0.83      1062
           3       0.91      0.92      0.92      2113

    accuracy                           0.89      5561
   macro avg       0.89      0.88      0.88      5561
weighted avg       0.89      0.89      0.89      5561

